In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

train=train.drop(['id'],axis=1)
test=test.drop(['id'],axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
train['Brand'].value_counts().plot(kind='bar')

In [ ]:
material_counts = pd.crosstab(train["Brand"], train["Material"])

# Plot a stacked bar chart
material_counts.plot(kind="bar", stacked=True, figsize=(8, 5), colormap="viridis")

In [ ]:
material_counts = pd.crosstab(train["Brand"], train["Size"])

# Plot a stacked bar chart
material_counts.plot(kind="bar", stacked=True, figsize=(8, 5), colormap="viridis")

In [ ]:
train.dropna()

In [ ]:
train['Material'].unique()

In [ ]:
train['Color'].unique()

In [ ]:
train['Style'].unique()

In [ ]:
train['Price'].plot.hist(bins=100)

In [3]:
train['Laptop Compartment']=train['Laptop Compartment'].replace(['No','Yes'],
                        [0, 1])
train['Waterproof']=train['Waterproof'].replace(['No','Yes'],
                        [0, 1])


<ipython-input-3-41ae9e46f185>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Laptop Compartment']=train['Laptop Compartment'].replace(['No','Yes'],
<ipython-input-3-41ae9e46f185>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Waterproof']=train['Waterproof'].replace(['No','Yes'],


In [4]:
train['Brand']=train['Brand'].replace(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma'],
                        [0, 1,2,3,4])
train['Color']=train['Color'].replace(['Black', 'Green', 'Red', 'Blue', 'Gray', 'Pink'],
                        [0, 1, 2, 3 ,4 ,5])
train['Style']=train['Style'].replace(['Tote', 'Messenger', 'Backpack'],
                        [0, 1,2])

<ipython-input-4-d315ccf2f513>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Brand']=train['Brand'].replace(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma'],
<ipython-input-4-d315ccf2f513>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Color']=train['Color'].replace(['Black', 'Green', 'Red', 'Blue', 'Gray', 'Pink'],
<ipython-input-4-d315ccf2f513>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.inf

In [5]:
train['Material']=train['Material'].replace(['Leather', 'Canvas', 'Nylon', 'Polyester'],
                        [0, 1,2, 3])
train['Size']=train['Size'].replace(['Small', 'Medium', 'Large'],
                        [0, 1,2])

<ipython-input-5-05893a5131f9>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Material']=train['Material'].replace(['Leather', 'Canvas', 'Nylon', 'Polyester'],
<ipython-input-5-05893a5131f9>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Size']=train['Size'].replace(['Small', 'Medium', 'Large'],


In [6]:
train

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0.0,0.0,1.0,7.0,1.0,0.0,0.0,0.0,11.611723,112.15875
1,0.0,1.0,0.0,10.0,1.0,1.0,1.0,1.0,27.078537,68.88056
2,1.0,0.0,0.0,2.0,1.0,0.0,1.0,2.0,16.643760,39.17320
3,2.0,2.0,0.0,8.0,1.0,0.0,1.0,1.0,12.937220,80.60793
4,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
299995,3.0,0.0,0.0,9.0,0.0,0.0,0.0,3.0,12.730812,129.99749
299996,0.0,0.0,2.0,6.0,0.0,1.0,0.0,3.0,26.633182,19.85819
299997,4.0,1.0,2.0,9.0,1.0,1.0,2.0,5.0,11.898250,111.41364
299998,3.0,2.0,0.0,1.0,0.0,1.0,0.0,5.0,6.175738,115.89080


In [ ]:
train=train.dropna()

In [7]:
from sklearn.impute import KNNImputer

In [11]:
rng = np.random.RandomState(0)

from sklearn.ensemble import RandomForestRegressor

# To use the experimental IterativeImputer, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

N_SPLITS = 4
regressor = RandomForestRegressor(random_state=0)


In [12]:
def get_scores_for_imputer(imputer, X_missing, y_missing):
    estimator = make_pipeline(imputer, regressor)
    impute_scores = cross_val_score(
        estimator, X_missing, y_missing, scoring="neg_mean_squared_error", cv=N_SPLITS
    )
    return impute_scores

In [13]:
def get_impute_knn_score(X_missing, y_missing):
    imputer = KNNImputer(missing_values=np.nan, add_indicator=True)
    knn_impute_scores = get_scores_for_imputer(imputer, X_missing, y_missing)
    return knn_impute_scores.mean(), knn_impute_scores.std()


a,b = get_impute_knn_score(
    X,y
)

KeyboardInterrupt: 

In [ ]:
imputer = KNNImputer(n_neighbors=2)
trainimp = imputer.fit_transform(train)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [8]:
X=train.drop(['Price'],axis=1)
y=train['Price']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
preds=lr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(preds,y_test,squared=False)

In [ ]:
test.isnull().sum()